<center><img src="https://news.illinois.edu/files/6367/543635/116641.jpg" alt="University of Illinois" width="250"/></center>

# Numerical Method in Finance

## Homework 02 ##

* Yu-Ching Liao <ycliao3@illinois.edu>

# Problem 01
Observe SPX option prices such as below.  Pick an expiration date.  Assume a reasonable interest rate(say 4%).  Index level can be observed at the top(say 3970).  Invert implied vols from your Black Scholes pricers for all the strikes in that expiration.  Hence you need to now code a root finding algorithm in Python which takes as inputs option prices from the market, and your BS pricer, find the implied volatility for the given strike and expiration. </br>
S&P 500 Index Options Prices - Barchart.com

## Basic Import

In [20]:
import math
import scipy.stats as si
import numpy as np
import pandas as pd

## Function Defination

In [21]:
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * si.norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Choose either 'call' or 'put'.")

    return price

def implied_volatility(S, K, T, r, market_price, option_type='call', initial_guess=0.2, max_iter=100, tol=1e-6):
    sigma = initial_guess
    for _ in range(max_iter):
        option_price = black_scholes(S, K, T, r, sigma, option_type)
        if abs(option_price - market_price) < tol:
            return sigma

        # Calculate option price derivative with respect to sigma (vega)
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        vega = S * si.norm.pdf(d1) * np.sqrt(T)

        if vega == 0:
            return sigma

        # Newton-Raphson method
        sigma -= (option_price - market_price) / vega

    return sigma

## Read Data

In [25]:
# set up date time
now = np.datetime64('2023-04-08')
expiration_date = np.datetime64('2023-08-18','D')

print("Observation stock Price: 3970")
print("Risk free rate: 4%")
print(f"T: {(expiration_date - now).astype(float) / 365.0}")
print("Observed at 2023-04-08")
print("Expiration date: 2023-08-18")

# read from csv
df = pd.read_csv('/Users/yu-chingliao/Library/CloudStorage/GoogleDrive-josephliao0127@gmail.com/My Drive/Note/UIUC/Spring_2023/IE525B_Numerical Method in Finance/Assignment /02/$spx-options-exp-2023-08-18-weekly-near-the-money-stacked-04-08-2023.csv')
df.head()

Observation stock Price: 3970
Risk free rate: 4%
T: 0.36164383561643837
Observed at 2023-04-08
Expiration date: 2023-08-18


,Strike,Moneyness,Bid,Midpoint,Ask,Last,Change,%Chg,Volume,Open Int,OI Chg,Volume Open Interest Ratio,IV,Type,Last Trade,Avg IV
0,"4,025.00",+1.95%,260.2,262.35,264.5,217.50,0.0,unch,0.0,5.0,unch,18.20%,Call,03/29/23,17.93%,NaN
1,"4,030.00",+1.83%,256.8,258.90,261.0,241.07,0.0,unch,0.0,20.0,unch,18.15%,Call,03/31/23,17.93%,NaN
2,"4,040.00",+1.58%,249.9,252.00,254.1,272.40,0.0,unch,0.0,1.0,unch,18.05%,Call,04/03/23,17.93%,NaN
3,"4,050.00",+1.34%,243.1,245.20,247.3,224.43,0.0,unch,0.0,82.0,unch,17.95%,Call,03/31/23,17.93%,NaN
4,"4,060.00",+1.10%,236.4,238.45,240.5,248.80,0.0,unch,0.0,5.0,unch,17.85%,Call,04/03/23,17.93%,NaN


In [26]:
# our observation
df['Observed Stock Price'] = 3970
df['Rf Rate'] = 0.04
df['T'] = (expiration_date - now).astype(float) / 365.0

# select only using col from the dataframe
df=df[['Observed Stock Price','Strike','T','Rf Rate','Midpoint','IV']]

# convert type to lower
df['IV']= df['IV'].str.lower()
df.dropna(inplace = True)

# Check datatype
df['Strike'] = df['Strike'].apply(lambda x: float(x.replace(',', ''))).astype(float)

market_data_lst = df.values.tolist()

implied_vols = []

for data in market_data_lst:
    iv = implied_volatility(*data)
    implied_vols.append(iv)
    S, K, T, r, market_price, option_type = data
    print(f"Option Type: {option_type} Strike Price: {K}, Option Price: {market_price}, Implied Volatility: {iv}")

Option Type: call Strike Price: 4025.0, Option Price: 262.35, Implied Volatility: 0.27438068453380005
Option Type: call Strike Price: 4030.0, Option Price: 258.9, Implied Volatility: 0.2731727514755915
Option Type: call Strike Price: 4040.0, Option Price: 252.0, Implied Volatility: 0.2707144332669891
Option Type: call Strike Price: 4050.0, Option Price: 245.2, Implied Volatility: 0.2683035305038347
Option Type: call Strike Price: 4060.0, Option Price: 238.45, Implied Volatility: 0.2658859104829015
Option Type: call Strike Price: 4070.0, Option Price: 231.75, Implied Volatility: 0.26346005595126704
Option Type: call Strike Price: 4075.0, Option Price: 228.45, Implied Volatility: 0.26227636378580904
Option Type: call Strike Price: 4080.0, Option Price: 225.1, Implied Volatility: 0.26102439993362264
Option Type: call Strike Price: 4090.0, Option Price: 218.6, Implied Volatility: 0.2586823349762912
Option Type: call Strike Price: 4100.0, Option Price: 212.15, Implied Volatility: 0.25632726

# Problem 02
Finish the derivation of Black-Scholes formula for pricing a compo option.

Let $ S $ be the foreign stock price, $ X$  be the strike price in domestic currency, $r$ be the domestic risk-free rate,$ q$ be the foreign risk-free rate, $σ_s$ be the volatility of the stock, and $σx$ be the volatility of the exchange rate. Let the correlation between the stock and the exchange rate be $ρ$. We'll use the Black-Scholes-Merton model to price this option. </br></br>
Let $C(S, X, T)$ be the price of a compo call option with time to expiration $T$. To derive the Black-Scholes formula for a compo option, we can follow these steps:

1. Define the risk-neutral measure.
2. Write down the stochastic differential equations (SDEs) for the stock price and exchange rate.
3. Derive the risk-neutral dynamics of the stock price and exchange rate.
4. Determine the dynamics of the stock price in domestic currency.
5. Apply Ito's Lemma to find the dynamics of the option price.
6. Solve the resulting partial differential equation (PDE) using risk-neutral pricing.

Describe: 

1. Risk-neutral measure:
In the risk-neutral world, the domestic and foreign risk-free rates are used to discount the expected payoffs. Therefore, the domestic risk-free rate, $r$, and the foreign risk-free rate, $q$, are the appropriate discount rates.

2. SDEs for the stock price and exchange rate:
Let $S_t$ be the foreign stock price, and let $X_t$ be the exchange rate (units of domestic currency per unit of foreign currency). Then, the SDEs for $S_t$ and $X_t$ are: 
### $dS_t =S_t(qdt+σ_sdW_{s,t})$ 
### $dX_t=X_t(rdt+σ_xdW_{x,t})$ 

where $W_{s,t}$ and $W_{x,t}$ are Wiener processes representing the stochastic components of the stock price and exchange rate, respectively, with correlation $\rho$: $dW_{s,t} dW_{x,t} = \rho dt$.

3. Risk-neutral dynamics:
Under the risk-neutral measure, the drift terms change. The risk-neutral SDEs for $S_t$ and $X_t$ become:
### $dS_t =S_t((r−q)dt+σ_sdW_{s,t}) $ 
### $dX_t=X_t(rdt+σ_xdW_{x,t})$ 

4. Dynamics of the stock price in domestic currency:
Now, we need to find the dynamics of the stock price in domestic currency, $S_t X_t$. Using Ito's product rule:

### $ d(S_tX _t​)=S_tdX_t+X_tdS_t+dS_tdX _t $
​
Substitute the risk-neutral SDEs of $S_t$ and $X_t$ into the equation above:
### $ d(S_tX _t​)=S_tX_t(rdt+σ_xdW_{x,t})+X_tS_t((r−q)dt+σ_sdW_{s,t})+ρσ_sσ_xS_tX_tdt $

5. Apply Ito's Lemma to the option price:
Using Ito's Lemma, we can find the dynamics of the option price, $C(S_t X_t, t)$:
###  $dC=$ $\frac{∂C}{∂t}$ $dt +$ $\frac{∂C}{∂(S_tX_t)}$ $d(S_tX_t) +$ $\frac{1}{2}$ $\frac{∂^2C}{∂(S_tX_t)^2}$ $(d(S_tX_t))^2$
Substitute the dynamics of $S_t Xt $ we derived in step 4 into the equation above:

#####  $dC=$ $\frac{∂C}{∂t}$ $dt +$ $\frac{∂C}{∂(S_tX_t)}$ $(S_tX_t(rdt+σ_xdW_{x,t}) +X_tS_t((r−q)dt+σ_sdW_{s,t})+ρσ_sσ_xS_tX_tdt)+$ $\frac{1}{2}$ $\frac{∂^2C}{∂(S_tX_t)^2}$ $(d(S_tX_t))^2$

Now we have the dynamics of the option price under the risk-neutral measure.

6. Solve the PDE using risk-neutral pricing:
The option price can be obtained by solving the following PDE:
### $\frac{∂C}{∂t}$ $+rS_tX_t$ $\frac{∂C}{∂(S_tX_t)}$ $+$ $\frac{1}{2}$ $(S_tX_t)^2(σ_s^2 + σ_x^2 + 2ρσ_sσ_x)$ $\frac{∂^2C}{∂(S_tX_t)^2}$ $-rC = 0$

with the boundary condition at expiration $T$:
### $ C(S_TX_T, T) = max(S_TX_T - K, 0) $
where $K$ is the strike price in domestic currency. This PDE is similar to the standard Black-Scholes PDE, but the volatility term is now the combined volatility of the stock and the exchange rate.

To solve this PDE, we can use a similar approach to the standard Black-Scholes formula. First, apply the following transformations:

### $ y=ln(S_t​X_t​),v(y,t)=e^{−αy−βt}C(S_t​X_t​,t)$

where $\alpha$ and $\beta$ are constants to be determined later. After transforming the PDE and applying the chain rule, we obtain a heat equation, which can be solved using standard methods. Finally, we can find the Black-Scholes formula for the compo option by applying the inverse transformations.

The resulting Black-Scholes formula for a compo call option is:
### $C(S_t​X_t​,t)=S_t​X_t​e^{−q(T−t)}N(d1​)−Ke^{−r(T−t)}N(d2​)$

where
### $d1​=​$ $\frac{​ln(\frac{S_t​X_t}​​{K})+(r−q+\frac{1}{2}​(σ_s^2​+σ_x^2​+2ρσ_s​σ_x​)(T−t))​}{\sqrt{(σ_s^2​+σ_x^2​+2ρσ_s​σ_x​)(T−t)}}​$
### $d2 = d1 - \sqrt{(σ_s^2​+σ_x^2​+2ρσ_s​σ_x​)(T−t)}$

and $N(\cdot)$ is the cumulative distribution function of the standard normal distribution.

A similar derivation can be done for a compo put option, which will yield the corresponding Black-Scholes formula for that case.

<center><img src="https://news.illinois.edu/files/6367/543635/116641.jpg" alt="University of Illinois" width="250"/></center>
